# PHYS 541 Assignment 4 Question 3

Beginning with the bikeshare_final.ipynb notebook in the Week7and8 folder, create a bikeshare system that models THREE locations (rather than just two).  Assuming the locations are called A, B, and C, use the following parameters:  total number of bikes = 100, p_AB = 0.3, p_BA=0.2, p_AC = 0.4, p_CA=0.1, p_BC=0.35, p_CB=0.15.  Try to determine the optimum initial configuration of the 100 bikes between the three locations such that the number of unsatisfied customers will be minimized over a three-hour period of bike sharing.

As a suggestion, try some configurations in steps of 10 bikes for the initial configuration, for example, try A = 10, B = 30, C = 60, and compare that to A = 10, B = 60, C = 30.  This may help you understand where the optimum configuration might be located in the (A,B,C) space.


In [1]:
from modsim import *

In [ ]:
def step(state, p_ab, p_ba, p_ac, p_ca, p_bc, p_cb):
    if flip(p_ab):
        bike_from_a_to_b(state)
        
    if flip(p_ba):
        bike_from_b_to_a(state)
        
    if flip(p_ac):
        bike_from_a_to_c(state)
        
    if flip(p_ca):
        bike_from_c_to_a(state)
    
    if flip(p_bc):
        bike_from_b_to_c(state)
        
    if flip(p_cb):
        bike_from_c_to_b(state)

def bike_from_a_to_b(state):
    if state.a == 0:
        state.a_empty += 1
        return
    state.a -= 1
    state.b += 1

def bike_from_b_to_a(state):
    if state.b == 0:
        state.b_empty += 1
        return
    state.b -= 1
    state.a += 1

def bike_from_c_to_b(state):
    if state.c == 0:
        state.c_empty += 1
        return
    state.c -= 1
    state.b += 1

def bike_from_a_to_c(state):
    if state.a == 0:
        state.a_empty += 1
        return
    state.a -= 1
    state.c += 1

def bike_from_b_to_c(state):
    if state.b == 0:
        state.b_empty += 1
        return
    state.b -= 1
    state.c += 1

def bike_from_c_to_a(state):
    if state.c == 0:
        state.c_empty += 1
        return
    state.c -= 1
    state.a += 1